<a href="https://colab.research.google.com/github/newton143/Econ8310/blob/master/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.express as px


In [6]:
# Define device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load training and test data using torchvision.datasets
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

# Create DataLoader objects
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.2MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 276kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.15MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 8.28MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [7]:
# Define the Neural Network
class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)  # Ensure correct input shape
        return self.model(x)


In [8]:
# Initialize model, loss, and optimizer
model = FashionNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [9]:
# Training Loop
def train(model, train_loader, loss_fn, optimizer, epochs=10):
    for epoch in range(epochs):
        for batch, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)  # Move data to the correct device
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                print(f"Epoch {epoch+1}, Batch {batch}, Loss: {loss.item():.4f}")
    print("Training complete!")


In [10]:
# Evaluate the model
def test(model, test_loader, loss_fn):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)  # Move data to the correct device
            pred = model(X).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [11]:
# Train and test
train(model, train_loader, loss_fn, optimizer, epochs=10)
test(model, test_loader, loss_fn)

Epoch 1, Batch 0, Loss: 2.3081
Epoch 1, Batch 100, Loss: 1.8482
Epoch 1, Batch 200, Loss: 1.1917
Epoch 1, Batch 300, Loss: 0.9264
Epoch 1, Batch 400, Loss: 0.7400
Epoch 1, Batch 500, Loss: 0.8010
Epoch 1, Batch 600, Loss: 0.7425
Epoch 1, Batch 700, Loss: 0.7333
Epoch 1, Batch 800, Loss: 0.6149
Epoch 1, Batch 900, Loss: 0.6402
Epoch 2, Batch 0, Loss: 0.5611
Epoch 2, Batch 100, Loss: 0.5837
Epoch 2, Batch 200, Loss: 0.6706
Epoch 2, Batch 300, Loss: 0.4422
Epoch 2, Batch 400, Loss: 0.5569
Epoch 2, Batch 500, Loss: 0.6381
Epoch 2, Batch 600, Loss: 0.3826
Epoch 2, Batch 700, Loss: 0.4735
Epoch 2, Batch 800, Loss: 0.4749
Epoch 2, Batch 900, Loss: 0.4559
Epoch 3, Batch 0, Loss: 0.5115
Epoch 3, Batch 100, Loss: 0.4284
Epoch 3, Batch 200, Loss: 0.5359
Epoch 3, Batch 300, Loss: 0.3751
Epoch 3, Batch 400, Loss: 0.3716
Epoch 3, Batch 500, Loss: 0.4424
Epoch 3, Batch 600, Loss: 0.4888
Epoch 3, Batch 700, Loss: 0.4129
Epoch 3, Batch 800, Loss: 0.5655
Epoch 3, Batch 900, Loss: 0.6311
Epoch 4, Batch 0

In [12]:
# Save model weights
torch.save(model.state_dict(), "fashion_mnist.pth")
print("Model weights saved!")

Model weights saved!


In [13]:
# Load and test the model using saved weights
def load_and_test():
    model = FashionNet().to(device)
    model.load_state_dict(torch.load("fashion_mnist.pth"))
    model.eval()
    print("Model loaded successfully!")
    test(model, test_loader, loss_fn)

load_and_test()

<ipython-input-13-e00fc5fefd13>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("fashion_mnist.pth"))


Model loaded successfully!
Test Accuracy: 85.42%
